In [1]:
import time
import pandas as pd
import json

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from math import ceil

In [2]:
BASE_URL = 'https://www.acra-ratings.ru'

In [46]:
def collect_data(web_element) -> None:
    soup = BeautifulSoup(web_element.get_attribute('outerHTML'), "html.parser")
    for row in soup.find_all(class_ = 'emits-row search-table-row'):
        temp = dict()
        columns = row.find_all(class_ ='emits-row__item')
        links = row.find_all('a')
        temp['name'] = columns[0].text.strip()
        temp['name_link'] = BASE_URL + links[0].get('href')
        temp['rating'] = columns[1].text.strip()
        temp['sector'] = columns[2].text.strip()
        temp['date'] = columns[3].text.strip()
        temp['press_release_link'] = BASE_URL + links[1].get('href')
        global data
        data.append(temp)

In [69]:
def get_text(web_element, idx) -> None:
    soup = BeautifulSoup(web_element.get_attribute('outerHTML'), "html.parser")
    paragraphs = soup.find_all('p')
    result = ''
    for paragraph in paragraphs:
        result += (paragraph.text.strip() + '\n')
    global data
    data[idx]['press_release_text'] = result

In [52]:
data = list()

In [53]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

browser = webdriver.Chrome(options=options)
browser.get('https://www.acra-ratings.ru/ratings/issuers/?text=&sectors[]=&activities[]=&countries[]=&forecasts[]\
        =&on_revision=0&rating_scale=106&rate_from=1&rate_to=23&page=1&sort=&count=10&')
time.sleep(2)

pages_quantity = browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[3]/div/div[1]/div[1]')
pages_quantity = ceil(int(pages_quantity.text.split()[-1]) / 10)

for page in tqdm(range(1, pages_quantity+1)):
    browser.get(f'https://www.acra-ratings.ru/ratings/issuers/?text=&sectors[]=&activities[]=&countries[]=&forecasts[]\
        =&on_revision=0&rating_scale=106&rate_from=1&rate_to=23&page={page}&sort=&count=10&')
    time.sleep(3)
    table = browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[3]/div/div[2]/div[1]/div[1]/div/div')
    collect_data(table)

browser.quit()


Starting...
##########################################
Parsing has finished


In [73]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

browser = webdriver.Chrome(options=options)

for i in tqdm(range(len(data))):
    browser.get(data[i]['press_release_link'])
    time.sleep(1)
    text = browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div/div[1]/div[2]/div')
    get_text(text, i)

browser.quit()

100%|██████████| 413/413 [37:11<00:00,  5.40s/it]


In [76]:
data_json = json.dumps(data, indent = 4) 

In [77]:
df = pd.read_json(data_json)

In [78]:
df.sample(5)

,name,name_link,rating,sector,date,press_release_link,press_release_text
175,"ПАО ""ЯТЭК""",https://www.acra-ratings.ru/ratings/issuers/256/,A(RU)\nСтабильный,Корпоративный сектор,6 апр 2023,https://www.acra-ratings.ru/press-releases/3894/,Кредитный рейтинг ПАО «ЯТЭК» (далее — Компания...
266,"ПАО АКБ ""Металлинвестбанк""",https://www.acra-ratings.ru/ratings/issuers/128/,A-(RU)\nСтабильный,Кредитные организации,14 дек 2022,https://www.acra-ratings.ru/press-releases/3681/,Кредитный рейтинг ПАО АКБ «Металлинвестбанк» (...
58,ООО «ОМГ»,https://www.acra-ratings.ru/ratings/issuers/353/,Отозван,Корпоративный сектор,12 июл 2023,https://www.acra-ratings.ru/press-releases/4092/,Аналитическое Кредитное Рейтинговое Агентство ...
310,Госкорпорация «Росатом»,https://www.acra-ratings.ru/ratings/issuers/226/,AAA(RU)\nСтабильный,Корпоративный сектор,7 ноя 2022,https://www.acra-ratings.ru/press-releases/3588/,Кредитный рейтинг Государственной корпорации п...
162,Оренбургская область,https://www.acra-ratings.ru/ratings/issuers/126/,A+(RU)\nПозитивный,Регионы и муниципалитеты,21 апр 2023,https://www.acra-ratings.ru/press-releases/3923/,Кредитный рейтинг Оренбургской области (далее ...


In [79]:
df.to_csv('../data_output/akra_all.csv')